In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [4]:
data = pd.read_csv("Churn_Modelling.csv")
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns = onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis = 1), geo_encoded_df], axis = 1)

x = data.drop('Exited', axis = 1)
y = data['Exited']

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)

# Save encoders and scaler for later use

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)



In [5]:
# Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons = 32, layers = 1):
    model = Sequential()
    model.add(Dense(neurons, activation = 'relu', input_shape = (xtrain.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons, activation = 'relu'))

    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(optimizer = 'adam', loss = "binary_crossentropy", metrics = ['accuracy'])

    return model

In [6]:
# Create Keras classifier

model = KerasClassifier(build_fn=create_model, epochs = 50, batch_size = 10, verbose = 0)

C:\Users\Hp\AppData\Local\Temp\ipykernel_14988\611775443.py:3: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs = 50, batch_size = 10, verbose = 0)


In [7]:
# Define the grid search parameters

param_grid = {
    'neurons':[16, 32, 64, 128],
    'layers':[1, 2, 3],
    'epochs':[50, 100]
}

In [8]:
# Perform grid search

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv = 3)
grid_result = grid.fit(xtrain, ytrain)

# Print the best paramerters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.857999 using {'epochs': 50, 'layers': 1, 'neurons': 128}
